In [1]:
from sogaPreprocessor import *
from producecfg import *
from libSOGAsmoother import *

In [10]:
compiledFile=compile2SOGA('../programs/SOGA/Toy.soga')
cfg = produce_cfg(compiledFile)

In [11]:
cfg.node_list

{'entry': EntryNode<>,
 'state0': StateNode<state0,None,x=gm([1.0],[0.000000],[1.000000])>,
 'state1': StateNode<state1,None,y=x+gm([1.0],[1.000000],[1.000000])>,
 'state2': StateNode<state2,None,y=exp(x)>,
 'exit': ExitNode<>}

In [12]:
dist = start_SOGA(cfg)
dist

Dist<['x', 'y'],pi: tensor([[1.]])
mu: tensor([[0.0000, 1.6487]])
sigma: tensor([[[1.0000, 1.6487],
         [1.6487, 4.6708]]])>

In [ ]:
def compute_exp_mean(norm, i):
    """"
    Computes the expected value of exp(X_i) where X_i is the i-th component of a multivariate normal distribution.
    """ 
    mu = norm.mean.unsqueeze(0).clone()
    t = torch.zeros_like(mu)
    t[0, i] = 1.0
    mu[0, i] = torch.exp(mu @ t.T + 0.5 * ( t @ norm.covariance_matrix @ t.T))
    return mu.squeeze(0)

def compute_exp_cov(norm, i):
    """
    Computes the covariance matrix of exp(X_i) where X_i is the i-th component of a multivariate normal distribution.
    """
    sigma = norm.covariance_matrix.clone()
    mean_i = compute_exp_mean(norm, i)[i]
    sigma[i, :] = sigma[:, i] = (norm.mean + sigma[:, i])*torch.exp(norm.mean[i] + 0.5*norm.covariance_matrix[i,i]) - norm.mean*mean_i
    mu = norm.mean.unsqueeze(0)
    t = torch.zeros_like(mu)
    t[0, i] = 2.0
    sigma[i, i] = torch.exp(mu @ t.T + 0.5 * ( t @ norm.covariance_matrix @ t.T)) - mean_i**2
    return sigma

mean = torch.tensor([0., 1.])
cov = torch.tensor([[1., 1.], [1., 2.]])
norm = torch.distributions.MultivariateNormal(mean, cov)
print(compute_exp_mean(norm, 0))
print(compute_exp_cov(norm, 0))

# These results are correct, but here I am just exponentiating x, without assigning it to y

tensor([1.6487, 1.0000])
tensor([[4.6708, 1.6487],
        [1.6487, 2.0000]])
